In [ ]:
import random
import base64
import hashlib

: 

## 1. Diffie-Hellman Key Exchange Implementation

### Mathematical Foundations

Important constants
- \(p\) is a large prime (modulus)
- \(g\) is a primitive root modulo \(p\)

Key exchange process:
1. Alice generates private key \($a \in [2, p-2]$\)
2. Alice computes public key \($A = g^a \mod p$\)
3. Bob generates private key \($b \in [2, p-2]$\)
4. Bob computes public key \($B = g^b \mod p$\)
5. Shared secret: \(s = $A^b \mod p = B^a \mod p$\)



## Our code requires a few helper functions as defined below

In [ ]:
def is_prime(n: int, k: int = 5) -> bool:
    """https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test#Complexity"""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False

    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

The is_prime function acts to define whether or not a number **could** be prime. The above implementation is based on the Rabin primality test, which is based on Fermats little theorum. These tests go deep into depth, but are not the primary focus of this notebook.

In [ ]:
def generate_large_prime(bits: int = 1024) -> int:
    """Generate probable prime with specified bit length"""
    while True:
        p = random.getrandbits(bits)
        p |= (1 << (bits - 1)) | 1
        print(p)
        if is_prime(p):
            return p

The generate_large_prime function generates a potential prime that is of power two, which is calculated using bit shifts. We require large prime numbers to act as our modulus

In [ ]:
def find_primitive_root(p: int) -> int:
    """Find smallest primitive root modulo prime p"""
    if p == 2:
        return 1
    
    factors = set()
    phi = p - 1
    n = phi
    d = 2
    while d * d <= n:
        while n % d == 0:
            factors.add(d)
            n //= d
        d += 1
    if n > 1:
        factors.add(n)

    for g in range(2, p):
        if all(pow(g, phi // factor, p) != 1 for factor in factors):
            return g
    raise ValueError("Primitive root not found")

The primitive root is used as our g value. It is the basis for the shared secret between Alice and Bob and ensures that their secrets have a common derviation. In class, we used a 2 as a common g value. In instances where we want other primitive roots, the above code can be used to help find them based some p value.

In [ ]:
class DiffieHellman:
    def __init__(self, key_bits: int = 2048, p: int = None, g: int = None):
        self.p = p if p else generate_large_prime(key_bits)
        self.g = g if g else find_primitive_root(self.p)
        self.private_key = random.randint(2, self.p - 2)
        self.public_key = pow(self.g, self.private_key, self.p)

    def generate_shared_secret(self, other_public: int) -> int:
        """Compute shared secret using peer's public key"""
        if not 2 <= other_public <= self.p - 2:
            raise ValueError("Invalid public key")
        return pow(other_public, self.private_key, self.p)

The DiffieHellman class is used to encapsulate each user. It abstracts the creation of shared secrets and stores the modulus, primitive root, public, and private keys. It is also used to calculate the shared secret between two users.

## 2. Base64 Encoding Utilities

For safe transmission of binary data over text-based channels

In [ ]:
def int_to_b64(num: int) -> str:
    """Convert integer to Base64 URL-safe string"""
    byte_length = (num.bit_length() + 7) // 8
    return base64.urlsafe_b64encode(num.to_bytes(byte_length, 'big')).decode()

def b64_to_int(b64_str: str) -> int:
    """Convert Base64 string back to integer"""
    return int.from_bytes(base64.urlsafe_b64decode(b64_str), 'big')

## 3. Stream Cipher Implementation

Using SHAKE128 extendable-output function (XOF) for arbitrary-length keystream

In [ ]:
def generate_keystream(shared_secret: int, length: int) -> bytes:
    """Generate pseudorandom keystream from shared secret"""
    secret_bytes = shared_secret.to_bytes((shared_secret.bit_length() + 7) // 8, 'big')
    return hashlib.shake_128(secret_bytes).digest(length)

def xor_encrypt(plaintext: bytes, keystream: bytes) -> bytes:
    """XOR plaintext with keystream"""
    return bytes(pt ^ ks for pt, ks in zip(plaintext, keystream))

def encrypt_message(message: str, shared_secret: int) -> str:
    """Encrypt message to Base64 ciphertext"""
    plainbytes = message.encode('utf-8')
    keystream = generate_keystream(shared_secret, len(plainbytes))
    cipherbytes = xor_encrypt(plainbytes, keystream)
    return base64.urlsafe_b64encode(cipherbytes).decode()

def decrypt_message(ciphertext: str, shared_secret: int) -> str:
    """Decrypt Base64 ciphertext to plaintext"""
    cipherbytes = base64.urlsafe_b64decode(ciphertext)
    keystream = generate_keystream(shared_secret, len(cipherbytes))
    plainbytes = xor_encrypt(cipherbytes, keystream)
    return plainbytes.decode('utf-8')

## 4. Example Usage

### Key Exchange Demonstration

In [ ]:
# Alice's setup
alice = DiffieHellman(key_bits=256, p=2**255 - 19, g = 2)  # using 25519 for p, 2 for g as placeholders
print(f"Alice's public key as an integer: {alice.public_key}")
print(f"Alice's public key in base 64: {int_to_b64(alice.public_key)}")

# Bob's setup (using Alice's p and g)
bob = DiffieHellman(p=alice.p, g=alice.g)
print(f"Bob's public key as an integer: {bob.public_key}")
print(f"Bob's public key in base 64: {int_to_b64(bob.public_key)}")

# Secret computation
alice_secret = alice.generate_shared_secret(bob.public_key)
bob_secret = bob.generate_shared_secret(alice.public_key)

print(f"\nShared secret match: {alice_secret == bob_secret}")
print(f"Shared secret value: {alice_secret}")

### Encryption/Decryption Demonstration

Alice wants to share Bob a message. She will encrypt her message with her shared secret. She will then send the ciphertext to Bob. The cyphertext seems nearly random.

In [ ]:
message = "This is a super super secret message that no one should see!"

ciphertext = encrypt_message(message, alice_secret) 
print(f"\nEncrypted ciphertext: {ciphertext}")

: 

Bob can use his shared secret to decrypt the cyphertext that Alice had sent him. 

In [ ]:
decrypted = decrypt_message(ciphertext, bob_secret)
print(f"\nDecrypted message: {decrypted}")

Alice and Bob both have the exact same shared secret, however this shared secret was never shared explcitly. They were able to derive this shared secret solely based on each other's public keys. As such, both Alice and Bob can communicate securely both ways.